In [1]:
import pandas as pd

## Overview

Task of finding nearest neighbours is very common. You can think of applications like finding duplicate or similar documents, audio/video search. Although using brute force to check for all possible combinations will find the exact nearest neighbour, but it is hardly scalable. Approximate algorithms accomplishing this task has been an area of active research. Locality sensitive hashing *LSH* is one such algorithms' family.

*LSH* refers to a family of functions to hash data points into buckets so that data points near each other are located in the same buckets with high probability, while data points far from each other are likely to be in different buckets.

![image info](./static/lsh-overview.png)

Real-world applications:
* Near-duplicate detection: *LSH* is commonly used to deduplicate large quantities of documents, webpages, and other files.
* Genome-wide association study: identify similar gene expressions in genome databases.
* Large-scale image search: Google used *LSH* along with PageRank to build image search technology VisualRank.
* Audio/video fingerprinting: In multimedia technologies, *LSH* is widely used as a fingerprinting technique A/V data.

This presentation will describe *LSH* class in general and *MinHashing* in particular

## Jaccard Similarity (also Jaccard Index)

The *Jaccard Similarity* is an **intersection** over a **union**.

Example:
* Who was the first ruler of England
* Who was the first king of England

<img src="./static/venn-diagram-1.png"  width="60%" height="60%">

Stats:
* size of intersection = 6
* size of union = 8
* **Jaccard Similarity** = 6 / 8 = 0.75

## Scale of the problem

First, we need to define a method of determining whether a document is a duplicate of another. We can use *string distance* metrics (Levenshtein, Jaro-Winkler, Jaccard Similarity) and compare word-by-word, document-by-document, and declare documents with number beyond a certain threshold as "matching".

The number of comparisons required is given by the following formula, which is pronounced "N-choose-2"
![N-choose-2 image](./static/n-choose-2-eq.png)

Let's assume we have a collection of 1 million documents, and that on average, a computer can calculate the *Jaccard Similarity* between two documents in *1ms*. (i.e. converting structured documents into set of words, comparing them discarding document structure)

The rough number of comparisons required:

![image info](./static/1m-doc-comparisons.png)

Next, the amount of time required:
![image info](./static/1m-doc-comparisons-time.png)

**16 years** of compute time! RAM served separately.

## A better way

The *MinHash* algorithm will provide us with a fast approximation to the *Jaccard Similarity* between two documents.

Input documents:
* Who was the first ruler of England
* Who was the first king of England
* Who was the last pharaoh of Egypt

And their *Jaccard Similarities*:

```shell
J("Who was the first king of England",  "Who was the first ruler of England") = 0.75
J("Who was the first king of England",  "Who was the last pharaoh of Egypt")  = 0.4
J("Who was the first ruler of England", "Who was the last pharaoh of Egypt")  = 0.4
```

To calculate MinHash we need to create the dictionary (a set of all words) from all our documents. Then, create a random permutation:

```python
("last", "Who", "Egypt", "king", "ruler", "was", "of", "England", "pharaoh", "the", "first")
```

and iterate over the rows, writing the index in the respective cell, if the word is present in the sentence.

In [2]:
pd.read_csv('./static/permutation_1.csv', header=0, dtype=str, keep_default_na=False, index_col='index')

,word,Who was the first king of England,Who was the first ruler of England,Who was the last pharaoh of Egypt
index,,,,
1,last,,,1
2,Who,2,2,2
3,Egypt,,,3
4,king,4,,
5,ruler,,5,
6,was,6,6,6
7,of,7,7,7
8,England,8,8,
9,pharaoh,,,9


Only the first word occurrence is relevant (giving a minimal index — hence the name *MinHash* ). We have a minimum value for all our documents and the first part of the fingerprint: *2 2 1*. To get the second one we need to create another random permutation and retrace our steps:


In [3]:
pd.read_csv('./static/permutation_2.csv', header=0, dtype=str, keep_default_na=False, index_col='index')

,word,Who was the first king of England,Who was the first ruler of England,Who was the last pharaoh of Egypt
index,,,,
1,the,1,1,1
2,of,2,2,2
3,England,3,3,
4,was,4,4,4
5,first,5,5,
6,ruler,,6,
7,Who,7,7,7
8,Egypt,,,8
9,pharaoh,,,9


Second part of the fingerprint is: *1 1 1*

In [4]:
pd.read_csv('./static/permutation_3.csv', header=0, dtype=str, keep_default_na=False, index_col='index')

,word,Who was the first king of England,Who was the first ruler of England,Who was the last pharaoh of Egypt
index,,,,
1,first,1,1,
2,king,2,,
3,Egypt,,,3
4,was,4,4,4
5,Who,5,5,5
6,of,6,6,6
7,pharaoh,,,7
8,last,,,8
9,England,9,9,


Third part of the fingerprint is:  *1 1 3*

Then, repeat permuting and searching as many times as big we want our fingerprint. For the purpose of example my consists of 6 items. We’ve already had 3, so let’s create 3 more permutations:

```python
("ruler", "king", "England", "Who", "the", "pharaoh", "of", "first", "Egypt", "last", "was")
("king", "England", "ruler", "last", "pharaoh", "the", "Who", "Egypt", "first", "of", "was")
("the", "pharaoh", "Who", "ruler", "England", "Egypt", "king", "last", "was", "first", "of")
```
producing following parts of the fingerprint:
* *2 1 4*
* *1 1 4*
* *1 1 1*

Completed MinHashes are:
```sql
MinHash("Who was the first king of England")  = [2, 1, 1, 2, 1, 1]
MinHash("Who was the first ruler of England") = [2, 1, 1, 1, 1, 1]
MinHash("Who was the last pharaoh of Egypt")  = [1, 1, 3, 4, 4, 1]
```

Now we can check how similar are two MinHashes by calculating **their** *Jaccard Similarities*:
```sql
MinHashSimilarity("Who was the first king of England", "Who was the first ruler of England") =
J(
  [2, 1, 1, 2, 1, 1],
  [2, 1, 1, 1, 1, 1]
) = 5/6 ≈ 0.83

MinHashSimilarity("Who was the first king of England", "Who was the last pharaoh of Egypt")  =
J(
  [2, 1, 1, 2, 1, 1],
  [1, 1, 3, 4, 4, 1]
) = 2/6 ≈ 0.33

MinHashSimilarity("Who was the first ruler of England", "Who was the last pharaoh of Egypt") =
J(
  [2, 1, 1, 1, 1, 1],
  [1, 1, 3, 4, 4, 1]
) 2/6 ≈ 0.33
```

which are close to *Jaccard Similarities* of the respective documents, and the more permutations we do, the closer the approximations get. How is that possible?

## More permutations mean better approximations

```python
with legend:
    A: 'word is present in both documents'
    B: 'word is present in one of them'
    C: 'word is in the dictionary, but in neither of the documents'
```

In [5]:
df = pd.read_csv('./static/permutation_abc.csv', header=0, dtype=str, keep_default_na=False, index_col='index')
df.style.hide(axis='index')

word,Who was the first king of England,Who was the first ruler of England
first,A,A
king,B,B
Egypt,C,C
was,A,A
Who,A,A
of,A,A
pharaoh,C,C
last,C,C
England,A,A
ruler,B,B


We can write the formula for the *Jaccard Similarity* as

```
J = a / (a + b)
```
where `a` is a number of rows of type `A` and `b` of type `B`

Since we have random permutations, let’s calculate the probability that two documents will have an equal fingerprint component. We can skip type `C` rows since they do not interfere in any way with a component value calculation (If we consider only two documents). So what is the probability that we will pick type `A` row, from the set of `A` and `B` rows?

```
P = a / (a + b)
```

which is exactly the same as *Jaccard Similarity*! That explains why our approximations were close and why more permutations mean better approximations.

## Hash part of MinHash

We now have an algorithm which could potentially perform better, but the more documents the bigger the dictionary, and thus the higher the cost of creating permutations, both in time and hardware. Instead of creating `n` permutations we can take a hash function ( `md5`, `sha256`, `crc32`, etc) use it on every word in a document and record a **minimal hash value**. It will be the first element of the fingerprint, then we will take another hash function, and so on until we have our `n` elements in the fingerprint.

`Q`:  Why does it work?

`A`:  Let’s consider what a permutation does — it basically maps each word from dictionary to a different number. The fact that mapped numbers are integers increased by one is not important to us.
      With a certain simplification - this is what a hash function does as well - **it maps a string to a number**, so basically the same as the permutation above!

Benefits of using hash functions:
* No need to keep the whole dictionary as before
* MinHash is easily calculated for new document/words
* No need to scan the whole dictionary for each document and create a permutation of the whole dictionary

In result, we are saving a lot of computational time.

So while we can now compute a fingerprint and compare it easily, we still need to **compare every fingerprint with every other**

## Banding

Let’s look back at our MinHashes, and **group** (also **band**) them by three elements:

```sql
MinHash("Who was the first king of England")  = [2, 1, 1, 2, 1, 1] => [211, 211]
MinHash("Who was the first ruler of England") = [2, 1, 1, 1, 1, 1] => [211, 111]
MinHash("Who was the last pharaoh of Egypt")  = [1, 1, 3, 4, 4, 1] => [113, 441]
```

We can see that our duplicates (two "matching" documents) have one band *211* in common, while unique have no common bands.

Let's compute the probability of at least one common band for duplicates:
* We want two documents (D1 & D2) with 75% *Jaccard Similarity* to be hashed in the same bucket for at least 1 of 2 bands
`Jaccard Similarity = P1 = 0.75`
* Probability that **(D1 & D2) are identical in a particular band** = `P2 = P1 to number-of-elements-in-band power = 0.75³ = 0.421875`
* Probability that **(D1 & D2) are different in all 2 bands** = `P3 = 1-P2 = 0.578125`
* Probability that **(D1 & D2) are not similar in all 2 bands** = `P4 = P3 to number-of-bands power = P3² = 0.334228516`
* Probability that **at least one band from (D1 & D2) will be common** = `P5 = 1 — P4 = 0.665771484`


So general equation will look like this: `P5 = 1 — (1 — J^n)^b`

Where:  
  `J` — Jaccard Similarity  
  `n` — number of elements in group  
  `b` — number of bands  

NOTE: In the scenario above we have ~33.4% chance of a `false negative` @ 75% similar documents.

We could also make `3` groups with `2` elements each:
 ```sql
MinHash("Who was the first king of England")  = [2, 1, 1, 2, 1, 1] => [21, 12, 11]
MinHash("Who was the first ruler of England") = [2, 1, 1, 1, 1, 1] => [21, 11, 11]
MinHash("Who was the last pharaoh of Egypt")  = [1, 1, 3, 4, 4, 1] => [11, 34, 41]
```

Here is the plot:
![image info](./static/lsh-minhash-bucketing.png)


Let’s analyze it:
For the pair with `J = 0.5` and `b = 3`, there is only `0.25` probability that we would find it as a duplicate.
However, with `b = 3` there is about `0.6` probability that we would mark it as a duplicate.

It means that in practice:
* More **false positives** for larger values of `b`
* More **false negatives** for smaller values of `b`

In other words - as almost everything in Data Science, LSH is sensitive to parameter selection.

Let's create association between single groups and the documents:

```
MinHash("Who was the first king of England")  = [2, 1, 1, 2, 1, 1] => [211, 211]
MinHash("Who was the first ruler of England") = [2, 1, 1, 1, 1, 1] => [211, 111]
MinHash("Who was the last pharaoh of Egypt")  = [1, 1, 3, 4, 4, 1] => [113, 441]
```
↓
```
[
211 => “Who was the first king of England”,
211 => “Who was the first ruler of England”,
113 => “Who was the last pharaoh of Egypt”
]
```
And
```
[
211 => “Who was the first king of England”,
111 => “Who was the first ruler of England”,
441 => “Who was the last pharaoh of Egypt”
]
```

**Banding** allows us to replace O(n²) search with O(n) Hashtable lookups!

![image info](./static/lsh-canonical-bucketing.png)

## Tweaking

Until this time, we learned that LSH takes 3 arguments:
* `k` — number of elements in `MinHash`  
    In our case, `k` was 6: for instance - [2, 1, 1, 2, 1, 1]
* `n` — number of elements in band  
    In our case, `n` was 3: for instance - [211, 211]
* `b` — number of bands and `b*n` must equal `k`  
    In our case, `b` was 2: for instance - [xxx, xxx]

Our goal is to find duplicates that have `Jaccard Similarity >= 0.17` (as `1-0.83`) so our probability chart should look like this:

![probability chart - single](./static/lsh-step-function-1.png)

We can only approximate to this behaviour by manipulating 3 parameters:

![probability chart - single](./static/lsh-step-function-multiple.png)

As we can see `b = 100 & n = 2` or `b = 50 & n = 4` are the ones closest to reference.
But how to get `200` hash functions?

## Hashing functions family

For an integer `x`:
![compatible hashing function family](./static/random-hash-eq.png)
Where the coefficients:
* `a` and `b` are randomly chosen integers less than `max(x)`.
* `c` is a prime number slightly bigger than `max(x)`.

Alternatively:
* use `MurmurHash` with `200` different seeds

## Shingling

In practice, individual words and dictionaries are replaced with `shingles`, where a `shingle` can be:
* Substrings of three-words, or five-words, allowing to retain more document structure
* Each document is converted into a set of characters of length `k` (also known as `k-shingles` or `k-grams`). Whitespaces are considered a valid character, such that relocation of paragraphs affect only a small number of shingles.
    For text processing the common `k` is between 8 and 40.

## Canonical workflow

![canonical workflow from Stanford](./static/lsh-canonical-workflow.png)

References:
* https://medium.com/@hbrylkowski/locality-sensitive-hashing-explained-304eb39291e4
* https://chrisjmccormick.wordpress.com/2015/06/12/minhash-tutorial-with-python-code/
* https://github.com/chrisjmccormick/MinHash/blob/master/runMinHashExample.py
* https://www.youtube.com/watch?v=e8dA0tscrCM
* https://towardsdatascience.com/understanding-locality-sensitive-hashing-49f6d1f6134